In [4]:
import pydrake

# 更新约束条件 / 代价函数

通常情况下, 在解决优化问题后, 我们希望稍微调整其约束条件和代价函数, 然后解决新的问题. 比如在模型预测控制中, 在每个时间实例中, 我们解决一个新的优化问题, 其约束/成本与前一时间实例中的略有不同. 我们可以更新旧 `MathematicalProgram` 对象中的约束条件 / 代价函数, 然后解决新的问题, 而不是构造新的 `MathematicalProgram` 对象. 

## 更新线性代价函数

对于线性代价函数 $a^Tx + b$, 我们可以调用 [`LinearCost.UpdateCoefficients()`](https://drake.mit.edu/pydrake/pydrake.solvers.html#pydrake.solvers.LinearCost.UpdateCoefficients) 来更新系数 `a` 和偏置 `b`

In [5]:
from pydrake.solvers import MathematicalProgram, Solve
import numpy as np

prog = MathematicalProgram()
x = prog.NewContinuousVariables(2)
cost1 = prog.AddLinearCost(2 * x[0] + 3 * x[1] + 2)
print(f"original cost: {cost1}")
prog.AddBoundingBoxConstraint(-1, 1, x)
result = Solve(prog)
print(f"optimal solution: {result.GetSolution(x)}")
print(f"original optimal cost: {result.get_optimal_cost()}")

# New cost: 3x[0] - 4x[1] + 5
cost1.evaluator().UpdateCoefficients(new_a=[3, -4], new_b=5)
print(f"updated cost: {cost1}")
# Solve the optimization problem again with the updated cost.
result = Solve(prog)
print(f"updated optimal solution: {result.GetSolution(x)}")
print(f"updated optimal cost: {result.get_optimal_cost()}")

original cost: LinearCost (2 + 2 * x(0) + 3 * x(1))
optimal solution: [-1. -1.]
original optimal cost: -3.0
updated cost: LinearCost (5 + 3 * x(0) - 4 * x(1))
updated optimal solution: [-1.  1.]
updated optimal cost: -2.0


## 更新二次型代价函数

形如 $\frac{1}{2}x^TQx + b'x + c$ 的代价函数, 我们可以调用 [`QuadraticCost.UpdateCoefficients`](https://drake.mit.edu/pydrake/pydrake.solvers.html#pydrake.solvers.QuadraticCost.UpdateCoefficients) 更新 $Q, b, c$ 

In [6]:
prog = MathematicalProgram()
x = prog.NewContinuousVariables(2)
cost1 = prog.AddQuadraticCost(x[0]**2 + 2 * x[1]**2 + x[0]*x[1] + 3*x[0] + 5)
print(f" original cost: {cost1}")
cost1.evaluator().UpdateCoefficients(new_Q=[[1., 2], [2., 4]], new_b=[1, 2.], new_c= 4)
print(f" updated cost: {cost1}")

 original cost: QuadraticCost (5 + 3 * x(0) + (x(0) * (x(0) + 0.5 * x(1))) + (x(1) * (0.5 * x(0) + 2 * x(1))))
 updated cost: QuadraticCost (4 + x(0) + 2 * x(1) + (x(0) * (0.5 * x(0) + x(1))) + (x(1) * (x(0) + 2 * x(1))))


## 更新边界条件

形如 $lower \le f(x) \le upper$ 的边界条件, 可以调用

- [`Constraint.UpdateLowerBound(new_lb)`](https://drake.mit.edu/pydrake/pydrake.solvers.html#pydrake.solvers.LinearConstraint.UpdateLowerBound) 更新下界 `new_lb`.
- [`Constraint.UpdateUpperBound(new_ub)`](https://drake.mit.edu/pydrake/pydrake.solvers.html#pydrake.solvers.LinearConstraint.UpdateUpperBound) 更新上界 `new_ub`.
- [`Constraint.set_bounds(new_lb, new_ub)`](https://drake.mit.edu/pydrake/pydrake.solvers.html#pydrake.solvers.LinearConstraint.set_bounds) 同时更新上下界

In [7]:
prog = MathematicalProgram()
x = prog.NewContinuousVariables(2)
constraint1 = prog.AddLinearConstraint(x[0] + 3 * x[1] <= 2)
print(f"original constraint: {constraint1}")
constraint1.evaluator().UpdateLowerBound([-1])
print(f"updated constraint: {constraint1}")
constraint1.evaluator().UpdateUpperBound([3])
print(f"updated constraint: {constraint1}")
constraint1.evaluator().set_bounds(new_lb=[-5], new_ub=[10])
print(f"updated constraint: {constraint1}")

original constraint: LinearConstraint
-inf <= (x(0) + 3 * x(1)) <= 2

updated constraint: LinearConstraint
-1 <= (x(0) + 3 * x(1)) <= 2

updated constraint: LinearConstraint
-1 <= (x(0) + 3 * x(1)) <= 3

updated constraint: LinearConstraint
-5 <= (x(0) + 3 * x(1)) <= 10



## 更新约束条件

形如 $lower \le Ax \le upper$ 的线性不等式约束, 可以调用 [`LinearConstraint.UpdateCoefficients(new_A, new_lb, new_ub)`](https://drake.mit.edu/pydrake/pydrake.solvers.html#pydrake.solvers.LinearConstraint.UpdateCoefficients) 

形如 $Ax = b$ 的线性等式约束, 可以调用 [`LinearEqualityConstraint.UpdateCoefficients(Aeq, beq)`](https://drake.mit.edu/pydrake/pydrake.solvers.html#pydrake.solvers.LinearEqualityConstraint.UpdateCoefficients) 

In [8]:
prog = MathematicalProgram()
x = prog.NewContinuousVariables(2)
linear_constraint = prog.AddLinearConstraint(3 * x[0] + 4 * x[1] <= 5)
linear_eq_constraint = prog.AddLinearConstraint(5 * x[0] + 2 * x[1] == 3)
print(f"original linear constraint: {linear_constraint}")
linear_constraint.evaluator().UpdateCoefficients(new_A = [[1, 3]], new_lb=[-2], new_ub=[3])
print(f"updated linear constraint: {linear_constraint}")

print(f"original linear equality constraint: {linear_eq_constraint}")
linear_eq_constraint.evaluator().UpdateCoefficients(Aeq=[[3, 4]], beq=[2])
print(f"updated linear equality constraint: {linear_eq_constraint}")

original linear constraint: LinearConstraint
-inf <= (3 * x(0) + 4 * x(1)) <= 5

updated linear constraint: LinearConstraint
-2 <= (x(0) + 3 * x(1)) <= 3

original linear equality constraint: LinearEqualityConstraint
(5 * x(0) + 2 * x(1)) == 3

updated linear equality constraint: LinearEqualityConstraint
(3 * x(0) + 4 * x(1)) == 2

